<a href="https://colab.research.google.com/github/HWMV/OrchestrAI/blob/main/frontend_doc/front_end_sample2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Code Structure ASCII**

orchestrai/
│
├── lib/
│   ├── main.dart
│   │
│   ├── screens/
│   │   └── crew_screen.dart
│   │
│   ├── widgets/
│   │   ├── component_sidebar.dart
│   │   ├── crew_canvas.dart
│   │   └── parameter_sidebar.dart
│   │
│   ├── models/
│   │   └── crew_model.dart
│   │
│   └── services/
│       └── api_service.dart
│
├── pubspec.yaml
│
└── test/
    └── widget_test.dart

## main.dart : 진입점, 전체 테마

In [ ]:
import 'package:flutter/material.dart';
import 'screens/crew_screen.dart';

void main() {
  runApp(OrchestrAIApp());
}

class OrchestrAIApp extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'OrchestrAI',
      theme: ThemeData(
        primarySwatch: Colors.blue,
        visualDensity: VisualDensity.adaptivePlatformDensity,
      ),
      home: CrewScreen(),
    );
  }
}

## screens/crew_screen.dart :
메인 화면, 3개의 주요 부분 (좌측 사이드바/중앙 캔버스/우측 사이드바)

## 왼쪽 사이드 바 (컴포넌트 목록, 서랍형태,, 펼쳐보기 기능으로 사이드 바 추가)

## 화면 추가 main screen > agent 생성 화면 (task, tool) : 우측 사이드바 (파라미터 입력)

In [ ]:
import 'package:flutter/material.dart';
import 'package:provider/provider.dart';
import '../models/crew_model.dart';
import '../widgets/crew_table.dart';
import '../widgets/component_sidebar.dart';
import 'agent_screen.dart';

class CrewScreen extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('OrchestrAI')),
      body: Row(
        children: [
          Expanded(
            flex: 1,
            child: ComponentSidebar(isDraggable: true),
          ),
          Expanded(
            flex: 4,
            child: Consumer<CrewModel>(
              builder: (context, crewModel, child) {
                return Column(
                  children: [
                    TextField(
                      decoration: InputDecoration(labelText: 'Crew Name'),
                      onChanged: (value) => crewModel.setCrewName(value),
                    ),
                    Expanded(
                      child: CrewTable(
                        onAgentTap: (int index) {
                          Navigator.push(
                            context,
                            MaterialPageRoute(
                              builder: (context) => AgentScreen(agentIndex: index),
                            ),
                          );
                        },
                      ),
                    ),
                  ],
                );
              },
            ),
          ),
        ],
      ),
    );
  }
}

## screens/agent_screen.dart

: Agent 조립 화면

In [ ]:
import 'package:flutter/material.dart';
import 'package:provider/provider.dart';
import '../models/crew_model.dart';
import '../widgets/component_sidebar.dart';
import '../widgets/lego_agent_builder.dart';
import '../widgets/parameter_sidebar.dart';

class AgentScreen extends StatelessWidget {
  final int agentIndex;

  AgentScreen({required this.agentIndex});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('Build Agent ${agentIndex + 1}')),
      body: Row(
        children: [
          Expanded(
            flex: 1,
            child: ComponentSidebar(isDraggable: true),
          ),
          Expanded(
            flex: 3,
            child: LegoAgentBuilder(agentIndex: agentIndex),
          ),
          Expanded(
            flex: 1,
            child: ParameterSidebar(agentIndex: agentIndex),
          ),
        ],
      ),
    );
  }
}

## widgets/component_sidebar.dart

## : 좌측 컴포넌트 사이드 바 (펼쳐보기 기능 추가), 2개의 screen에 추가

## 수정 : main screen에서 의자 위젯(버트) Agent add 기능 추가

In [ ]:
import 'package:flutter/material.dart';
import 'package:provider/provider.dart';
import '../models/crew_model.dart';

class ComponentSidebar extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return Consumer<CrewModel>(
      builder: (context, crewModel, child) {
        return Container(
          color: Colors.grey[200],
          child: ListView(
            children: [
              ListTile(
                title: Text('Agents (${crewModel.agents.length})'),
                trailing: Icon(Icons.add),
                onTap: () => _showAgentDialog(context, crewModel),
              ),
              ListTile(
                title: Text('Tasks (${crewModel.tasks.length})'),
                trailing: Icon(Icons.add),
                onTap: () => _showTaskDialog(context, crewModel),
              ),
              ListTile(
                title: Text('Tools'),
                trailing: Icon(Icons.add),
                onTap: () => _showToolDialog(context, crewModel),
              ),
            ],
          ),
        );
      },
    );
  }

  void _showAgentDialog(BuildContext context, CrewModel crewModel) {
    showDialog(
      context: context,
      builder: (BuildContext context) {
        String agentName = '';
        return AlertDialog(
          title: Text('Add New Agent'),
          content: TextField(
            decoration: InputDecoration(hintText: "Enter agent name"),
            onChanged: (value) => agentName = value,
          ),
          actions: [
            TextButton(
              child: Text('Cancel'),
              onPressed: () => Navigator.of(context).pop(),
            ),
            TextButton(
              child: Text('Add'),
              onPressed: () {
                if (agentName.isNotEmpty) {
                  crewModel.addAgent(AgentModel(
                    role: agentName,
                    goal: '',
                    backstory: '',
                    tools: [],
                  ));
                  Navigator.of(context).pop();
                }
              },
            ),
          ],
        );
      },
    );
  }

  void _showTaskDialog(BuildContext context, CrewModel crewModel) {
    // Similar implementation to _showAgentDialog
    // but for adding a new task
  }

  void _showToolDialog(BuildContext context, CrewModel crewModel) {
    // Similar implementation to _showAgentDialog
    // but for adding a new tool
  }
}

## widgents/crew_table.dart

In [ ]:
import 'package:flutter/material.dart';

class CrewTable extends StatelessWidget {
  final Function(int) onAgentTap;

  CrewTable({required this.onAgentTap});

  @override
  Widget build(BuildContext context) {
    return GridView.builder(
      gridDelegate: SliverGridDelegateWithFixedCrossAxisCount(
        crossAxisCount: 2,
        childAspectRatio: 1.5,
      ),
      itemCount: 4,
      itemBuilder: (context, index) {
        return GestureDetector(
          onTap: () => onAgentTap(index),
          child: Card(
            child: Center(
              child: Text('Agent ${index + 1}'),
            ),
          ),
        );
      },
    );
  }
}

## widgents/lego_agent_builder.dart

In [ ]:
import 'package:flutter/material.dart';
import 'package:provider/provider.dart';
import '../models/crew_model.dart';

class LegoAgentBuilder extends StatelessWidget {
  final int agentIndex;

  LegoAgentBuilder({required this.agentIndex});

  @override
  Widget build(BuildContext context) {
    return Consumer<CrewModel>(
      builder: (context, crewModel, child) {
        final agent = crewModel.agents[agentIndex];
        return Column(
          children: [
            LegoPartSelector(
              label: 'Head (Agent)',
              onSelected: (value) {
                agent.name = value;
                crewModel.updateAgent(agentIndex, agent);
              },
            ),
            LegoPartSelector(
              label: 'Body (Task)',
              onSelected: (value) {
                agent.task = value;
                crewModel.updateAgent(agentIndex, agent);
              },
            ),
            // Add LegoPartSelector for tools (hands, feet)
          ],
        );
      },
    );
  }
}

class LegoPartSelector extends StatelessWidget {
  final String label;
  final Function(String) onSelected;

  LegoPartSelector({required this.label, required this.onSelected});

  @override
  Widget build(BuildContext context) {
    return Column(
      children: [
        Text(label),
        ElevatedButton(
          onPressed: () {
            // Show dialog to select Lego part
            showDialog(
              context: context,
              builder: (context) => AlertDialog(
                title: Text('Select $label'),
                content: TextField(
                  onSubmitted: (value) {
                    onSelected(value);
                    Navigator.pop(context);
                  },
                ),
              ),
            );
          },
          child: Text('Select'),
        ),
      ],
    );
  }
}

## widgets/crew_canvas.dart
: 중앙 캔버스

## Agent 생성 화면의 중앙 화면 (component 세팅하는 화면)

## Agent 생성 화면 좌측 화면은 Agent 조립 이미지 추가

In [ ]:
import 'package:flutter/material.dart';
import 'package:provider/provider.dart';
import '../widgets/component_sidebar.dart';
import '../widgets/crew_canvas.dart';
import '../widgets/parameter_sidebar.dart';
import '../models/crew_model.dart';
import '../services/api_service.dart';

class CrewScreen extends StatelessWidget {
  final ApiService apiService = ApiService();

  @override
  Widget build(BuildContext context) {
    return ChangeNotifierProvider(
      create: (_) => CrewModel(),
      child: Scaffold(
        appBar: AppBar(title: Text('OrchestrAI')),
        body: Row(
          children: [
            Expanded(flex: 1, child: ComponentSidebar()),
            Expanded(flex: 3, child: CrewCanvas()),
            Expanded(flex: 1, child: ParameterSidebar()),
          ],
        ),
        floatingActionButton: Consumer<CrewModel>(
          builder: (context, crewModel, child) {
            return FloatingActionButton(
              onPressed: () async {
                try {
                  final result = await apiService.createWorkflow(crewModel.toJson());
                  ScaffoldMessenger.of(context).showSnackBar(
                    SnackBar(content: Text('Workflow created successfully')),
                  );
                } catch (e) {
                  ScaffoldMessenger.of(context).showSnackBar(
                    SnackBar(content: Text('Failed to create workflow: $e')),
                  );
                }
              },
              child: Icon(Icons.send),
            );
          },
        ),
      ),
    );
  }
}

## widgets/parameter_sidebar.dart

: 우측 파라미터 사이드 바

## Agent 생성 화면에서 우측 사이드 바로 변경

In [ ]:
import 'package:flutter/material.dart';

class ParameterSidebar extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return Container(
      color: Colors.grey[200],
      child: ListView(
        children: [
          TextField(
            decoration: InputDecoration(
              labelText: 'Agent Name',
            ),
          ),
          TextField(
            decoration: InputDecoration(
              labelText: 'Task Description',
            ),
          ),
          TextField(
            decoration: InputDecoration(
              labelText: 'Tool Name',
            ),
          ),
        ],
      ),
    );
  }
}


## models/crew_model.dart

: Crew, Agent, Task들의 상태를 관리하기 위한 모듈

In [ ]:
import 'package:flutter/foundation.dart';

class CrewModel extends ChangeNotifier {
  List<AgentModel> agents = [];
  List<TaskModel> tasks = [];

  void addAgent(AgentModel agent) {
    agents.add(agent);
    notifyListeners();
  }

  void addTask(TaskModel task) {
    tasks.add(task);
    notifyListeners();
  }

  Map<String, dynamic> toJson() {
    return {
      'agents': agents.map((agent) => agent.toJson()).toList(),
      'tasks': tasks.map((task) => task.toJson()).toList(),
    };
  }
}

class AgentModel {
  String role;
  String goal;
  String backstory;
  List<String> tools;

  AgentModel({required this.role, required this.goal, required this.backstory, required this.tools});

  Map<String, dynamic> toJson() {
    return {
      'role': role,
      'goal': goal,
      'backstory': backstory,
      'tools': tools,
    };
  }
}

class TaskModel {
  String description;
  String expectedOutput;
  String agentName;

  TaskModel({required this.description, required this.expectedOutput, required this.agentName});

  Map<String, dynamic> toJson() {
    return {
      'description': description,
      'expected_output': expectedOutput,
      'agent_name': agentName,
    };
  }
}

## service/api_service.dart

: 백엔드 서버와의 호출을 연결하기 위한 모듈

## 토요일 미팅 시 한솔님 백엔드 서버 URL로 연동

In [ ]:
import 'package:http/http.dart' as http;
import 'dart:convert';

class ApiService {
  final String baseUrl = 'https://your-backend-url.com/api';

  Future<Map<String, dynamic>> createWorkflow(Map<String, dynamic> data) async {
    final response = await http.post(
      Uri.parse('$baseUrl/create-workflow'),
      headers: {'Content-Type': 'application/json'},
      body: json.encode(data),
    );

    if (response.statusCode == 200) {
      return json.decode(response.body);
    } else {
      throw Exception('Failed to create workflow');
    }
  }
}